# Introduction to Artificial Intelligence
Laboratory 1



Exercise 1 Route searching 
1.	Create a set of cities (as points) with coordinates x, y on a plane with height as z coordinate. The cost of going from city A to city B is equal to the Euclidean distance between two cities, if there exists a road. You should define scenarios according to two criteria: 
a.	There are all the direct connections / c.a. 80% of possible connections
b.	The problem is symmetrical / asymmetrical (in asymmetrical – going up is height +10%, going down: -10%)
You should choose the coordinates randomly from the range <-100, 100> for x,y and <0, 50> for z.
2.	Represent the created map as a weighted (directed) graph, where cities are the nodes and roads are the edges of the graph.
3.	In the created scene, solve the traveling salesman problem: The salesman starts from a chosen city and has to visit every city exactly once before returning to the starting city. The goal is to find a path with the lowest cost.
In the problem, we define state as a partial or full path from the starting city and the corresponding state. You should represent the search problem in a form of state tree.
a.	Implement a full search of the tree, using BFS and DFS methods.
b.	Approximate the solution using greedy search (NN and Dijkstra)
c.	Solve/approximate the solution using A* with inadmissible/admissible heuristics
d.	Approximate the solution using ACO algorithm
4.	Test each algorithm, in each scenario, for n=5…20 cities, in terms of the found path cost, time and memory consumption.


In [1]:
import random
from support import euclidean_distance,euclidean_distance_assymetrical
class CitiesMap():
    def __init__(self,n,seed = None):
        """
        initialize with number of cities to include in graph 
        """    
        
        if seed is not None:
            random.seed(seed)
        self.seed = seed
        self.n = n 
        self.cities = []
        self.routes = [[0 for _ in range(n)] for _ in range(n)]
        # we can represent routes between cities as a matrix with n x n shape 
        for i in range(n):
            self.cities.append((random.randint(-101,101),random.randint(-101,101),random.randint(0,51)))

        # now let's make sure that we don't have duplicated cities so we'll get derired number of unique cities
        
        len_check = len(set(self.cities))
        
        while (len_check != n):
            self.cities.append((random.randint(-101,101),random.randint(-101,101),random.randint(0,51)))
            len_check = len(set(self.cities))
        self.cities = list(set(self.cities))
    
    
    def calculate_distance(self,symmetrical: bool = True , connections: float = 1):
        """
        Fill matrix with distnaces between city i and j  0 < i,j < n with 0 on main diagonal , conenctions 1 means all cities will be connected
        0.5 only half of them and so on 
        
        """
             
        if (symmetrical):
            if (connections==1):   
                for i in range(self.n):
                    for j in range(i+1 , self.n):  # to not double calcualte
                        
                        distance = euclidean_distance(self.cities[i], self.cities[j])
                    
                        # Store the route only once, as it is undirected
                        self.routes[i][j] = distance
                        self.routes[j][i] = distance
                return self.routes
            else:
                # in this case we nned to delete number of connections that equals to connections * (n^2-n)/2 
                
                n_to_delete = int(round((1 - connections) * ((self.n**2 - self.n) / 2),0))

                indexes_list = []    
                                
                for i in range(self.n):
                    for j in range(i+1 , self.n):  # to not double calcualte
                        
                        indexes_list.append((i,j))
                        
                        distance = euclidean_distance(self.cities[i], self.cities[j])
                    
                        # Store the route only once, as it is undirected
                        self.routes[i][j] = distance
                        self.routes[j][i] = distance
            
                indexes_to_delete = random.sample(indexes_list,n_to_delete)
                
                for i in indexes_to_delete:
                    # in the list that we iterate over we have pair of matrix indexes when let's say i means position with 0 index in matrix rows, j column
                    self.routes[i[0]][i[1]] = 0     # deletes i,i
                    self.routes[i[1]][i[0]] = 0        # deletes j,i 
                    
                    
                
                
                return self.routes
                
                
                
        else: 
            if (connections==1):  
                for i in range(self.n):
                    for j in range(self.n):
                        self.routes[i][j] = euclidean_distance_assymetrical(self.cities[i], self.cities[j])
                        
                return self.routes  
            else:
                # in this case we nned to delete number of connections that equals to connections * (n^2-n) # assymetrical 
                
                n_to_delete = int(round((1 - connections) * ((self.n**2 - self.n) / 2),0))
                indexes_list = []    
                                
                for i in range(self.n):
                    for j in range(self.n):
                        # however with connections to delete we'll keep approach from previous one as I understand that lack connections means no road between two points 
                        # no matter how long 
                        
                        self.routes[i][j] = euclidean_distance_assymetrical(self.cities[i], self.cities[j])
                        
                        # adds to indexes base only ones from above main diagonal 
                        if (j>i):
                            indexes_list.append((i,j))
                    
                    
                        
                indexes_to_delete = random.sample(indexes_list,n_to_delete)
                
                
                for i in indexes_to_delete:
                    # in the list that we iterate over we have pair of matrix indexes 
                    self.routes[i[0]][i[1]] = 0     # deletes i,i
                    self.routes[i[1]][i[0]] = 0
                    
                    
                
                
                return self.routes
                
                
    
    


In [40]:
class RouteSearch:
    def __init__(self,cities_map : CitiesMap):
        self.nodes = cities_map.cities
        self.size = cities_map.n
        self.edges = cities_map.routes
        self.states = []
        
        # self.visited = {}
    def dfs_sort(self,visited,starting_node):
        
        visited.add(starting_node)

        # iterating over neighbours 
        for neighbour in range(self.size):
            
            if ((neighbour not in visited) & (self.edges[starting_node][neighbour] > 0)):   # to loop only through valid routes
                self.dfs(visited,neighbour)

    def dfs_all_paths(self, starting_node: int):
        """
        Perform DFS to explore all possible paths, both complete and incomplete.
        
        Parameters:
        starting_node : int - the node from which to start the search
        
        Returns:
        list : A list of tuples where each tuple is (path, cost). Path can be incomplete or complete.
        """
        # Stack for DFS: stores tuples (path, cost)
        stack = [([starting_node], 0)]
        all_paths = []

        while stack:
            path, current_cost = stack.pop()
            last_node = path[-1]

            # Record the current (incomplete or complete) path and its cost
            all_paths.append((path, current_cost))

            # If the path includes all cities, check if it can return to the start
            if len(path) == self.size:
                return_to_start_cost = self.edges[last_node][starting_node]
                if return_to_start_cost != 0:
                    total_cost = current_cost + return_to_start_cost
                    # Record the completed cycle (path returns to the start)
                    all_paths.append((path + [starting_node], total_cost))

            # Explore all unvisited neighbors (continue DFS)
            for neighbor in range(self.size - 1, -1, -1):  # reverse order to simulate stack behavior
                if neighbor not in path and self.edges[last_node][neighbor] != 0:
                    new_path = path + [neighbor]
                    new_cost = current_cost + self.edges[last_node][neighbor]
                    stack.append((new_path, new_cost))

        return all_paths

        
    
    

In [58]:
test = CitiesMap(5,seed=42)
test.calculate_distance(symmetrical=False,connections=1);

In [59]:
algos = RouteSearch(test)

In [60]:
algos.dfs_all_paths(0)


[([0], 0),
 ([0, 1], 203.73502398949475),
 ([0, 1, 2], 255.47630344796968),
 ([0, 1, 2, 3], 328.8904710179967),
 ([0, 1, 2, 3, 4], 412.6400978828371),
 ([0, 1, 2, 3, 4, 0], 554.07913068496),
 ([0, 1, 2, 4], 396.50126577733886),
 ([0, 1, 2, 4, 3], 480.0117441617907),
 ([0, 1, 2, 4, 3, 0], 668.9323624118906),
 ([0, 1, 3], 250.42720748951967),
 ([0, 1, 3, 2], 322.5107701095789),
 ([0, 1, 3, 2, 4], 463.5357324389481),
 ([0, 1, 3, 2, 4, 0], 604.974765241071),
 ([0, 1, 3, 4], 334.1768343543601),
 ([0, 1, 3, 4, 2], 473.7420162587305),
 ([0, 1, 3, 4, 2, 0], 669.7446693019992),
 ([0, 1, 4], 321.4013318265302),
 ([0, 1, 4, 2], 460.9665137309006),
 ([0, 1, 4, 2, 3], 534.3806813009276),
 ([0, 1, 4, 2, 3, 0], 723.3012995510276),
 ([0, 1, 4, 3], 404.9118102109821),
 ([0, 1, 4, 3, 2], 476.9953728310413),
 ([0, 1, 4, 3, 2, 0], 672.9980258743101),
 ([0, 2], 194.9549691595472),
 ([0, 2, 1], 245.93300364499987),
 ([0, 2, 1, 3], 292.6251871450248),
 ([0, 2, 1, 3, 4], 376.37481400986525),
 ([0, 2, 1, 3, 4,